In [2]:
import gym
from gym import spaces
import pygame
import random

class FlappyBirdEnv(gym.Env):
    def __init__(self):
        super(FlappyBirdEnv, self).__init__()

        self.YELLOW = pygame.Color(255,255,0)
        self.BLACK = pygame.Color(0, 0, 0)
        self.WHITE = pygame.Color(255, 255, 255)
        self.GREEN = pygame.Color(0, 255, 0)
        pygame.init()

        self.surfaceWidth = 800
        self.surfaceHeight = 500
        self.surface = pygame.display.set_mode((self.surfaceWidth, self.surfaceHeight))
        pygame.display.set_caption('Flappy Bird')
        self.clock = pygame.time.Clock()

        self.bird_width = 30
        self.bird_height = 30

        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(low=0, high=255, shape=(self.surfaceHeight, self.surfaceWidth, 3), dtype=int)

        self.x = 150
        self.y = 200
        self.y_move = 0

        self.x_block = self.surfaceWidth
        self.y_block = 0

        self.block_width = 50
        self.block_height = random.randint(0, self.surfaceHeight / 2)
        self.gap = self.bird_height * 5

        self.block_move = 4

        self.score = 0
        self.game_over = False

    def reset(self):
        self.x = 150
        self.y = 200
        self.y_move = 0

        self.x_block = self.surfaceWidth
        self.y_block = 0

        self.block_height = random.randint(0, self.surfaceHeight / 2)

        self.score = 0
        self.game_over = False

        return self._get_state()

    def step(self, action):
        if action == 1:
            self.y_move = -5

        self.y = self.y + self.y_move

        self.surface.fill(self.BLACK)
        self._draw_bird()
        self._draw_score()

        if 3 <= self.score < 5:
            self.block_move = 5

        if 5 <= self.score < 8:
            self.block_move = 6

        if 8 <= self.score < 14:
            self.block_move = 7

        if self.score >= 14:
            self.block_move = 8

        self._draw_blocks()
        self.x_block -= self.block_move

        if self.y > self.surfaceHeight - self.bird_height or self.y < 0:
            self.game_over = True

        if self.x_block < (-1 * self.block_width):
            self.x_block = self.surfaceWidth
            self.block_height = random.randint(0, self.surfaceHeight / 2)

        if self.x + self.bird_width > self.x_block and self.x < self.x_block + self.block_width:
            if self.y < self.block_height or self.y + self.bird_height > self.block_height + self.gap:
                self.game_over = True

        if self.x > self.x_block + self.block_width and self.x < self.x_block + self.block_width + self.bird_width / 4:
            self.score += 1

        pygame.display.update()
        self.clock.tick(80)

        return self._get_state(), self._get_reward(), self.game_over, {}

    def render(self, mode='human'):
        pass

    def close(self):
        pygame.quit()

    def _get_state(self):
        # Capture the current game screen as the state
        state = pygame.surfarray.array3d(pygame.display.get_surface())
        return state

    def _get_reward(self):
        # Define the reward logic based on the game state
        return self.score

    def _draw_bird(self):
        pygame.draw.rect(self.surface, self.YELLOW, [self.x, self.y, self.bird_height, self.bird_width])

    def _draw_score(self):
        font = pygame.font.Font('freesansbold.ttf', 20)
        text = font.render('Score:' + str(self.score), True, self.WHITE)
        self.surface.blit(text, [3, 3])

    def _draw_blocks(self):
        pygame.draw.rect(self.surface, self.GREEN, [self.x_block, self.y_block, self.block_width, self.block_height])
        pygame.draw.rect(self.surface, self.GREEN, [self.x_block, self.y_block + self.block_height + self.gap, self.block_width, self.surfaceHeight])
